In [ ]:
# !pip3 install git+https://github.com/openai/whisper.git
# !pip3 install ffmpeg

In [5]:
import os
import re
import whisper
import pandas as pd
# import ffmpeg

model = whisper.load_model("large")

def transcribe_Whisper_large(filename):
    result = model.transcribe(filename, language='zh')
    return result["text"]

df_whisper_large = pd.DataFrame({
    'Filename': [],
    'Start_time': [],
    'End_time': [],
    'Whisper_large': []
})

100%|██████████████████████████████████████| 2.88G/2.88G [00:23<00:00, 134MiB/s]


In [6]:
for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/conversationA/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Whisper_large(f'segmented_audio/conversationA/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_whisper_large.loc[len(df_whisper_large)] = {'Filename': f'D00{number}_A','Start_time': start_time,'End_time': end_time,'Whisper_large': predicted}

for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/conversationB/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Whisper_large(f'segmented_audio/conversationB/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_whisper_large.loc[len(df_whisper_large)] = {'Filename': f'D00{number}_B','Start_time': start_time,'End_time': end_time,'Whisper_large': predicted}

for i in range(1, 71):
    number = str(i).zfill(2)
    input_files = [f for f in os.listdir(f"segmented_audio/voice_command/00{number}")]
    for input_file in input_files:
        predicted = transcribe_Whisper_large(f'segmented_audio/voice_command/00{number}/{input_file}')
        match = re.search(r'(\d+\.\d+)_(\d+\.\d+)\.wav$', input_file)
        start_time = match.group(1)
        end_time = match.group(2)
        df_whisper_large.loc[len(df_whisper_large)] = {'Filename': f'P00{number}','Start_time': start_time,'End_time': end_time,'Whisper_large': predicted}

df_whisper_large.to_csv('predicted_transcription/Whisper-large.csv')

# EVALUATION

In [ ]:
!pip3 install jiwer
!pip3 install cer

In [2]:
import pandas as pd

df_whisper_large = pd.read_csv('predicted_transcription/Whisper-large.csv', index_col=0)
df_A_cleaned = pd.read_csv('ground_truth_cleaned/D00XX_A_ground_truth_cleaned.csv', index_col=0)
df_B_cleaned = pd.read_csv('ground_truth_cleaned/D00XX_B_ground_truth_cleaned.csv', index_col=0)
df_P_cleaned = pd.read_csv('ground_truth_cleaned/P00XX_ground_truth_cleaned.csv', index_col=0)
df_whisper_large = df_whisper_large.rename(columns={'Filename': 'file', 'Start_time': 'start_time', 'End_time': 'end_time'})
df_A_cleaned['file'] = df_A_cleaned['file'].str.extract(r'/(D\d{4}_A)', expand=False)
df_B_cleaned['file'] = df_B_cleaned['file'].str.extract(r'/(D\d{4}_B)', expand=False)
df_P_cleaned['file'] = df_P_cleaned['file'].str.extract(r'/(P\d{4})', expand=False)
df_merged = pd.merge(df_A_cleaned, df_whisper_large, on=['file', 'start_time', 'end_time'], how='outer')
df_merged = pd.merge(df_B_cleaned, df_merged, on=['file', 'start_time', 'end_time'], how='right', suffixes=('_x',''))
df_merged = df_merged.drop(columns=['participant_x', 'ground_truth_x', 'ground_truth_cleaned_x'])
df_merged = pd.merge(df_merged, df_P_cleaned, on=['file', 'start_time', 'end_time'], how='outer', suffixes=('','_x'))
df_merged['participant'] = df_merged['participant'].combine_first(df_merged['participant_x'])
df_merged['ground_truth'] = df_merged['ground_truth'].combine_first(df_merged['ground_truth_x'])
df_merged['ground_truth_cleaned'] = df_merged['ground_truth_cleaned'].combine_first(df_merged['ground_truth_cleaned_x'])
df_merged = df_merged.drop(columns=['participant_x', 'ground_truth_x', 'ground_truth_cleaned_x'])
# df_merged = df_merged.sort_values(by='start_time', ascending=True)


from jiwer import wer

WER = []
for i in range(df_merged.shape[0]):
    try:
        WER.append(wer(df_merged['ground_truth_cleaned'].iloc[i], df_merged['Whisper_large'].iloc[i]))
    except:
        WER.append('NaN')

df_merged['WER'] = WER

# !pip3 install cer

from cer import calculate_cer_corpus

# cer_corpus_score
# {
#     'count': 2,
#     'mean': 0.3127282211789254,
#     'median': 0.3127282211789254,
#     'std': 0.07561653111280243,
#     'min': 0.25925925925925924,
#     'max': 0.36619718309859156
# }


CER = []
for i in range(df_merged.shape[0]):
    try:
        refs = df_merged['ground_truth_cleaned'].iloc[i]
        hyps = df_merged['Whisper_large'].iloc[i]
        hyps = [sent.split() for sent in hyps]
        refs = [sent.split() for sent in refs]
        cer_corpus_score = calculate_cer_corpus(hyps, refs)
        CER.append(cer_corpus_score['mean'])
    except:
        CER.append('NaN')

df_merged['CER'] = CER
df_merged.head(20)

,file,start_time,end_time,participant,ground_truth,ground_truth_cleaned,Whisper_large,WER,CER
0,D0001_A,87.97,94.85,1.0,大/r[大]家好，我是叫<姓名>，那。,大家好，我是叫<姓名>，那。,大家好我是瑪那,1.0,NaN
1,D0001_A,95.58,102.83,1.0,我是，今年是，<年龄>岁来自<居住地>，呃/i。,我是，今年是，<年龄>岁来自<居住地>，。,我是今年是来自,1.0,NaN
2,D0001_A,104.09,105.68,1.0,我是。,我是。,我是,1.0,NaN
3,D0001_A,106.30,119.12,1.0,资深/p的口/b[口/r/b]吃患者。,资深的口吃患者。,名之身的口口吃患者,1.0,NaN
4,D0001_A,119.13,124.49,1.0,我是从小就有口吃。,我是从小就有口吃。,我是從小就有口吃,1.0,NaN
5,D0001_A,126.46,133.89,1.0,到[到]现在，嗯/i/p，一直。,到现在，，一直。,到到現在嗯一直,1.0,NaN
6,D0001_A,136.61,141.33,1.0,伴随我，到我现在那嗯/i。,伴随我，到我现在那。,放水我到现在那,1.0,NaN
7,D0001_A,142.72,149.50,1.0,现在我已经工/r/b作了，嗯/i/p。,现在我已经工作了，。,現在我已經工作了,1.0,NaN
8,D0001_A,150.85,154.01,1.0,那我是。,那我是。,这样过 evet,2.0,NaN
9,D0001_A,157.80,169.91,1.0,现在虽然工作了，但是还是会存在口吃的问题。,现在虽然工作了，但是还是会存在口吃的问题。,現在雖然已經工作了但是還是會存在口吃的問題,1.0,0.904762
